<a href="https://colab.research.google.com/github/omlad93/ArchLab/blob/main/Cepstrum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Seminar on DSP Algorithms and Applications**
### Introduction and Demonstaration of Cepstrum Analysis
> DR. Yakov Stein  
> Omri Elad  
> 06.03.2022   

In this notebook we will Explore the Fundemtals ideads and consepts 

### Initialzing notebook

In [36]:
!pip install acoustics > null

In [37]:
import matplotlib.pyplot as plt
import matplotlib.collections as collections
%matplotlib inline
plt.style.use('bmh')
import numpy as np
from math import pi,e,ceil
import soundfile as sf
from google.colab import drive
from scipy import signal
from scipy.io.wavfile import read as load_sound
import acoustics.cepstrum as cep

drive.mount('/content/gdrive', force_remount=True)
dsp_dir = "/content/gdrive/MyDrive/4th year/DSP Seminar"

Mounted at /content/gdrive


In [38]:
plt.rcParams['figure.figsize'] = [50, 18]
# Parameters
n = 20

In [39]:
def sine_wave(f0:float,t:np.ndarray, phase:float=0.0):
    return np.sin(2 * np.pi * f0 * t + phase)

def sum_of_sines(f0:float,t:np.ndarray, n:int):
    waveform = np.zeros((t.shape[0],), dtype=float)
    elements = [
                (f0 / f) * sine_wave(f0=f,t=t,phase=f)
                for f in [f0*i for i in range(1,n+1)]
                ]
    info = [
            (2*np.pi*f,f)
            for  f in [f0*i for i in range(1,n+1)]
    ]
    for ele in elements:
      waveform += ele
    return waveform , elements ,info

def convert_to_mel(frequencies):
  return 2410*(2410*np.log(1+frequencies/625))

def fourier(f,freq):
  fft = (np.abs(np.fft.rfft(np.hamming(len(f)) * f)))
  frequencies = np.fft.rfftfreq(len(f), 1/freq)
  return frequencies,fft

def liftering(signal,low=0,high=-1):
  for i in range(len(signal)):
    if i not in range(low,high+1):
      signal[i]=0
  return signal


## Step By Step Cepstrum Creation
### Creating Independt sine waves
generating sin(ωt+ω)

In [ ]:
FREQ = 1_000 # Hz
SAMPLES = ceil(1_000*4*pi) 
F0 = 0.1592
time= np.arange(SAMPLES) / FREQ


sum, all, info = sum_of_sines(F0 ,time, n)
fig, axes = plt.subplots(nrows= ceil(n//2), ncols=2)
fig.tight_layout(pad=3.0)
for i in range(len(all)):
  ax = axes[i//2][i%2]
  ax.plot(time,all[i] )
  ax.set_title(f'sin({info[i][0]:.0f}t+{info[i][1]:.2f}) in Time')
  ax.axhline(y=0, color='black', linestyle='-')
  ax.vlines(x=0,ymin=min(all[i]),ymax=max(all[i]), color='black', linestyle='-')

### Creation of wave with harmonics (sum of sine waves)
creating $f(t)=\sum_{i=1}^{n}sin(i𝜔_0t_𝜔)$  

In [ ]:
DURATION = 0.628
FREQ = 22_050
SAMPLES = int(FREQ*DURATION)
t = np.arange(SAMPLES) / FREQ
time= np.arange(SAMPLES) / FREQ
F0 =500


sum, all, info = sum_of_sines(F0 ,time, n)
fig, ax = plt.subplots()
ax.plot(time, sum)
ax.set_xlabel('time (s)')
ax.set_title('Harmonic (sum of sines) in Time')
ax.axhline(y=0, color='black', linestyle='-')
ax.vlines(x=0,ymin=min(sum)-1,ymax=max(sum)+1, color='black', linestyle='-')


### Fourier Transform
### Creating FT of Independt sine waves
generating $F(sin(i𝜔_0+𝜔))$ for range of 𝜔

In [ ]:
fourier_abs_sines = []
sum, all, info = sum_of_sines(F0 ,time, n)
fig, axes = plt.subplots(nrows=n//2, ncols=2,)

for i in range(len(all)):
  abs_fourier = np.abs(np.fft.fft(all[i]))
  f_axis = np.fft.fftfreq(SAMPLES)*2*pi
  fourier_abs_sines.append(abs_fourier)
  ax = axes[(i%(n//2))][(i//(n//2))]
  ax.plot(f_axis ,abs_fourier)
  ax.set_title(f'F(sin({info[i][0]:.1f}t+{info[i][1]:.0f})) in Frequency [Rad/s]')
  ax.tick_params(axis='x', which='major', labelsize=15)
  ax.axhline(y=0, color='black', linestyle='-')
  ax.vlines(x=0,ymin=0,ymax=max(abs_fourier)+1, color='black', linestyle='-')

plt.setp(axes , 
         xticks=      [-3.14 ,-1.57 ,0    , 1.57 , 3.14], 
         xticklabels= ['-π'  ,'-π/2' ,0 , 'π/2' , 'π'  ])
fig.tight_layout(pad=3.0)


### creating log(FT of sum of sines)
creating $F(\sum_{i=1}^{n}sin(i𝜔_0t_𝜔))$ and $log(F(\sum_{i=1}^{n}sin(i𝜔_0t_𝜔)))$

In [ ]:
frequencies = np.fft.fftshift(np.fft.fftfreq(sum.size)*2*pi)
fft = np.abs(np.fft.fft(np.hamming(sum.size) * sum))
log_fft_sq = np.log(np.square(fft))
fourier_of_sum = np.log(np.abs(fft))

fig, ax = plt.subplots(nrows=2)
plt.setp(axes , 
         xticks=      [-3.14 ,-1.57 ,0    , 1.57 , 3.14], 
         xticklabels= ['-π'  ,'-π/2' ,0 , 'π/2' , 'π'  ])
fig.suptitle('Fourier', fontsize=40)
ax[0].plot(frequencies, fft)
ax[0].set_xlabel('frequency (Rad/s)', fontsize=20)
ax[0].set_title('F(f(t))', fontsize=20)
ax[0].axhline(y=0, color='black', linestyle='-')
ax[0].vlines(x=0,ymin=min(fft),ymax=max(fft), color='black', linestyle='-')

ax[1].plot(frequencies, log_fft_sq)
ax[1].set_xlabel('frequency (Rad/s)', fontsize=20)
ax[1].set_title('log(F{f(t)}^2)', fontsize=20)
ax[1].axhline(y=0, color='black', linestyle='-')
ax[1].vlines(x=0,ymin=min(log_fft_sq)-1,ymax=max(log_fft_sq)+1, color='black', linestyle='-')



### creating Cepstrum
creating $F^{-1}(log(F(\sum_{i=1}^{n}sin(i𝜔_0t_𝜔)^2)))^2$  

In [ ]:
epsilon = 10e-4
sum, all, info = sum_of_sines(F0 ,time, n)

quefrencies = np.fft.fftfreq(fft.size)
sum_cepstrum,fs = cep.complex_cepstrum(sum)
# print(sum_cepstrum)
fig, ax = plt.subplots()
ax.plot(quefrencies, sum_cepstrum)
ax.set_xlabel('quefrency (s)', fontsize=20)
ax.set_title('C(f(t))', fontsize=20)
ax.set_xlim(-epsilon, max(quefrencies)+0.1)
ax.axhline(y=0, color='black', linestyle='-')
ax.vlines(x=0,ymin=min(sum_cepstrum),ymax=max(sum_cepstrum), color='black', linestyle='-')


## Testing Applicability
We will now see a feature of of cepstral analysis  
* simplication of convolution in time: $C(f(t)*h(t)) ≈ C(f(t))+C(h(t))$


In [ ]:
k,l = 100,500
SIG = np.pad(np.asarray([k/(n+k)*np.sin(n/5) for n in range(l)]),[10,0],'constant')
CON = np.convolve(SIG,SIG)
SIG_PADDED = np.pad(SIG, [0, CON.size - SIG.size],'constant') # Zero Padded in the End
t= np.arange(SIG_PADDED.size)

sig,_ = cep.complex_cepstrum(SIG)
fig, ax = plt.subplots(nrows=2,sharex=True)
ax[0].plot(t, SIG_PADDED)
ax[0].set_xlabel('Time (s)', fontsize=20)
ax[0].set_title('Pulses', fontsize=20)
ax[0].axhline(y=0, color='black', linestyle='-')
ax[0].vlines(x=0,ymin=min(SIG_PADDED),ymax=max(SIG_PADDED), color='black', linestyle='-')

ax[1].plot(t, CON)
ax[1].set_xlabel('Time (s)', fontsize=20)
ax[1].set_title('Pulses', fontsize=20)
ax[1].axhline(y=0, color='black', linestyle='-')
ax[1].vlines(x=0,ymin=min(CON),ymax=max(CON), color='black', linestyle='-')


In [ ]:
sig, _ = cep.complex_cepstrum(SIG_PADDED)
con, _ = cep.complex_cepstrum(CON)
f = np.fft.fftfreq(SIG_PADDED.size,1/(2*pi))
q = np.fft.fftshift(np.fft.fftfreq(f.size))
sum = sig+sig
fig, ax = plt.subplots(nrows=3)
plt.setp(ax , 
         xlabel = 'Cepstrum by quefrency')
ax[0].plot(q, sum)
ax[0].axhline(y=0, color='black', linestyle='-')
ax[1].plot(q, con)
ax[1].axhline(y=0, color='black', linestyle='-')
ax[2].plot(q, sum)
ax[2].plot(q, con, 'r+')
ax[2].axhline(y=0, color='black', linestyle='-')
